In [52]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Reshape, MultiHeadAttention
from tensorflow.keras.models import Model
import numpy as np

In [62]:
"""
tensorflow.keras.layers.MultiHeadAttention
+ assert key.shape[0]==value.shape[0]
+ output.shape = query.shape if output_shape==None else (query.shape[0],output_shape:int)
+ attentionMask.shape = (query.shape[0],key.shape[0])
"""

attentionMask = np.triu( np.ones((4,9),dtype=np.float32), k=0) # mask all i-j>k element
# k=0:upper; k>0:strict upper; k<0: strict down # some paper k=0 some k=1

q = Input(shape=(4,3)) # NLP:sentence(time),word(feature) # CV:W*H,channel
k = Input(shape=(9,5))
v = Input(shape=(9,6))
x = MultiHeadAttention(num_heads=2, key_dim=7, output_shape=10)(q,v,k,attention_mask=attentionMask)

model = Model([q,k,v], x)
model.summary()

Model: "model_20"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_60 (InputLayer)           [(None, 4, 3)]       0                                            
__________________________________________________________________________________________________
input_62 (InputLayer)           [(None, 9, 6)]       0                                            
__________________________________________________________________________________________________
input_61 (InputLayer)           [(None, 9, 5)]       0                                            
__________________________________________________________________________________________________
multi_head_attention_24 (MultiH (None, 4, 10)        388         input_60[0][0]                   
                                                                 input_62[0][0]            

In [63]:
for i in range(len(model.weights)):
    print( i, model.weights[i].name, model.weights[i].shape )

0 multi_head_attention_24/query/kernel:0 (3, 2, 7)
1 multi_head_attention_24/query/bias:0 (2, 7)
2 multi_head_attention_24/key/kernel:0 (5, 2, 7)
3 multi_head_attention_24/key/bias:0 (2, 7)
4 multi_head_attention_24/value/kernel:0 (6, 2, 7)
5 multi_head_attention_24/value/bias:0 (2, 7)
6 multi_head_attention_24/attention_output/kernel:0 (2, 7, 10)
7 multi_head_attention_24/attention_output/bias:0 (10,)


In [48]:
3*2*7 + 2*7 + 5*2*7 + 2*7 + 6*2*7 + 2*7 + 14*10 + 10

388